# Лабораторная работа №2

## Задание

Провести моделирование временного ряда из задачи 1 с помощью нейросетевых методов на свой выбор. Смоделировать ряд, сделав предсказание на разумный период. Провести анализ метрик. Изобразить график модели с предсказанием с учетом возможных ошибок. Сделать заключение, сравнив результаты